In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Dropout
from keras import layers
from keras.regularizers import l2,l1
from sklearn.model_selection import StratifiedKFold as kfold

In [ ]:
# Comment this part out if you want to mount the google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
data=pd.read_csv('/content/drive/MyDrive/Wlv/6CS012/garments_worker_productivity.csv')

In [ ]:
data

In [3]:
data=data.drop('date',axis=1)

In [ ]:
data.isna().sum()

In [4]:
data.dropna(inplace=True)

In [5]:
data.reset_index(drop=True,inplace=True)

In [6]:
X=data.drop('actual_productivity',axis=1)
y=data['actual_productivity']

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer([("t1", OneHotEncoder(), [1]),("t2", OrdinalEncoder(), [0,2])],remainder='passthrough')
X=ct.fit_transform(X)

In [ ]:
X

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X=scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X.shape

In [48]:
# Wihout regularisation or dropout
model=Sequential()
model.add(Dense(13, input_shape=(13,), activation='relu'))
model.add(Dense(9,activation='relu'))
model.add(Dense(9,activation='relu'))
model.add(Dense(1, activation='relu'))

In [10]:
# with dropout
DROP=0.05
model=Sequential()
model.add(Dense(13, input_shape=(13,), activation='swish'))
model.add(Dropout(DROP))
model.add(Dense(9,activation='swish'))
model.add(Dropout(DROP))
model.add(Dense(9,activation='swish'))
model.add(Dropout(DROP))
model.add(Dense(1, activation='swish'))

In [20]:
# with regularisation
model=Sequential()
model.add(Dense(13, input_shape=(13,), activation='relu',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(9,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(9,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='relu',activity_regularizer=l2(0.01)))

In [35]:
# with regularisation, another setup
model=Sequential()
model.add(Dense(13, input_shape=(13,), activation='swish',kernel_regularizer=l1(0.001)))
model.add(Dense(9,activation='swish',kernel_regularizer=l2(0.001)))
model.add(Dense(9,activation='swish',kernel_regularizer=l2(0.001)))
model.add(Dense(1, activation='swish'))

In [11]:
# Compiling without callback
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [ ]:
# Training without callback
history = model.fit(X_train, y_train, epochs=200, batch_size=50, validation_split=0.3)

In [12]:
# Defining callback
callback1 = keras.callbacks.EarlyStopping(monitor='val_loss', patience=60,min_delta=0.00001, restore_best_weights=True)

In [ ]:
# Training with callback
history = model.fit(X_train, y_train, epochs=1000, batch_size=50, validation_split=0.3, callbacks=[callback1])

In [ ]:
# Plotting the error
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Predicting
pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
# real value
print('\nThe R^2 score is {}'.format(r2_score(y_test,pred)))

# 1%